In [1]:
%matplotlib inline

import arcpy
import os, shutil
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.metrics import r2_score
from scipy import stats


workingDir = "Z:\Auxiliary\Project_Folders\CS_Sandbox\\NCALM\\bulking\\v2\\points_run0"
arcpy.env.workspace=workingDir
arcpy.env.overwriteOutput = 1


In [2]:
# Set up functions

# Remove outlier function
def rmout(data, nstd):
    s = np.std(data)
    outInd = (data<data.mean()+nstd*data.std()) & (data>data.mean()-nstd*data.std())
    return s, outInd

In [119]:
# Bring in dataframe from previous notebook
df = pd.read_pickle(os.path.join("Z:\\Auxiliary\\Project_Folders\\CS_Sandbox\\NCALM\\bulking\\v2\\pkl", "bulkingv2.pkl"))

In [120]:
# For proof of concept, grab two row
#dff = df[(df.df_id=="G29") | (df.df_id=="G30") ]
dff = df
dff

,df_id,el_min,el_max,da_at_toe,eros,dep,area_m2,num_izs,trunk_le,total_le,mass_bal,growthFac_tot,growthFac_tru,hl_tot,hl_tru
0,G28,316.635925,577.833069,4.864887e+05,-8181.928718,9305.394420,16472.358784,4,956.421997,1275.076294,1123.465702,6.416815,8.554727,0.204848,0.273098
1,G30,418.236359,699.475281,3.240192e+06,-10902.321104,7444.465534,9637.933766,3,792.861023,887.067322,-3457.855571,12.290297,13.750608,0.317043,0.354714
2,G03,453.847687,604.707642,6.033704e+05,-14258.675030,9932.677192,6850.263804,1,377.389008,377.389008,-4325.997839,37.782433,37.782433,0.399747,0.399747
3,G06,390.107239,539.656006,7.301279e+04,-1417.382198,2887.706650,3307.642483,5,324.997986,444.260681,1470.324452,3.190429,4.361203,0.336624,0.460153
4,G16,322.283264,464.893494,1.908886e+05,-1555.781638,2303.745875,4654.288570,2,449.580994,473.906708,747.964236,3.282886,3.460515,0.300925,0.317207
5,G21,320.917664,525.310669,1.180245e+05,-7466.337988,6564.654507,9720.523991,4,643.939026,743.997314,-901.683481,10.035437,11.594790,0.274723,0.317410
6,G22,319.939789,467.059570,4.776164e+04,-5947.453785,2672.737456,6402.602715,4,495.229004,625.546326,-3274.716328,9.507615,12.009502,0.235186,0.297074
7,G26,310.760498,529.360046,5.532677e+05,-6695.485281,6595.804865,16199.441124,2,1013.530029,1179.493042,-99.680416,5.676579,6.606104,0.185333,0.215681
8,P20,492.500000,705.460022,1.135930e+05,-3811.663818,1690.224731,3030.340492,2,407.958008,453.904114,-2121.439087,8.397509,9.343275,0.469174,0.522015
9,P22,534.635010,701.436646,1.410274e+04,-586.700806,776.953979,1530.707928,1,268.485992,268.485992,190.253174,2.185219,2.185219,0.621268,0.621268


# Confirm end of geometry is bottom of flow

In [10]:
#lines = 'Full_DF_Vector\df_lines'
lines = 'P03_smooth'
dxPts = os.path.join("Z:\\Auxiliary\\Project_Folders\\CS_Sandbox\\NCALM\\bulking\\", "points_10m_p03.shp")
dx = 10

In [17]:
## Generate points along each trunk
# Select trunks
#where = """init_id LIKE '%_T' """
#arcpy.management.SelectLayerByAttribute(lines, "NEW_SELECTION", where, None)
# Generate pts
arcpy.GeneratePointsAlongLines_management(lines, dxPts, "DISTANCE", "{0} Meters".format(dx), None, "END_POINTS")
# Add field for distance along line
arcpy.AddField_management(dxPts, "dx", "FLOAT")
# Add fields for cardinal direction along flow at given point
arcpy.AddField_management(dxPts, "card", "FLOAT")



<Result 'Z:\\Auxiliary\\Project_Folders\\CS_Sandbox\\NCALM\\bulking\\points_10m_p03.shp'>

In [18]:
# # Populate fields for distance along track and cardinal direction
# Bring in as dataframe
df_point_data = arcpy.da.TableToNumPyArray(dxPts, ["FID", "init_id", "dx", "card", "SHAPE@X", "SHAPE@Y"])
dfp = pd.DataFrame(df_point_data)
dfp.rename(columns={'SHAPE@X': 'x', 'SHAPE@Y': 'y'}, inplace=True)


# Initialize for looping through debris flows
dfs = dfp.init_id.unique()
nx = -1

# Process in dataframe
for d in dfs:
    # Assign all in increments of dx
    dfp.loc[dfp.init_id == d, 'dx'] = (dfp[dfp.init_id == d].index - nx - 1) * dx
    # Reset number of points
    nx = np.max(dfp[dfp.init_id == d].index)
    # Reset final value to = trunk length
    #mx = dff[dff.df_id == d[:3]].trunk_le.values[0]
    mx = 772.77099609
    dfp.iloc[nx, dfp.columns.get_loc('dx')] = mx
    # Populate the cardinal direction
    x1 = dfp.x.iloc[:-1].values
    x2 = dfp.x.iloc[1:].values
    y1 = dfp.y.iloc[:-1].values
    y2 = dfp.y.iloc[1:].values
    bearing = np.degrees(np.arctan2(x2-x1,y2-y1))
    dfp.iloc[:-1, dfp.columns.get_loc('card')] = bearing
    # Reset final point in each track to NaN
    dfp.iloc[nx, dfp.columns.get_loc('card')] = np.nan


# Add data to shapefile

    
    
    
dfp

,FID,init_id,dx,card,x,y
0,0,P03_T,0.000000,121.921867,382341.570000,3.899138e+06
1,1,P03_T,10.000000,121.562614,382350.039168,3.899132e+06
2,2,P03_T,20.000000,129.201141,382358.556850,3.899127e+06
3,3,P03_T,30.000000,140.307571,382366.299398,3.899121e+06
4,4,P03_T,40.000000,147.652466,382372.668741,3.899113e+06
...,...,...,...,...,...,...
73,73,P03_T,730.000000,161.496231,382772.961108,3.898693e+06
74,74,P03_T,740.000000,168.437103,382776.130076,3.898684e+06
75,75,P03_T,750.000000,147.727798,382778.133429,3.898674e+06
76,76,P03_T,760.000000,92.914680,382783.385116,3.898666e+06


In [19]:
# Loop through and generate lines orthogonal to flow
#arcpy.management.BearingDistanceToLine(in_table, out_featureclass, x_field, y_field, distance_field, {distance_units}, bearing_field, {bearing_units}, {line_type}, {id_field}, {spatial_reference}, {attributes})

# Generate table for input
lt = pd.DataFrame()

lt["x"] = dfp.x
lt["y"] = dfp.y
lt["dist"] = 30
lt["bear1"] = dfp.card+90
lt["bear2"] = dfp.card-90
#lt["id"] = dfp.init_id + "_" + round(dfp.dx,0).astype(int).astype(str)
lt["id"] = dfp.init_id + "_" + [s.zfill(3) for s in round(dfp.dx,0).astype(int).astype(str)]

# Drop NA, export to CSV
lt.dropna(inplace=True)              
ltCSV = os.path.join(workingDir, 'lt.csv')
lt.to_csv(ltCSV, index=False)  

# Get spatial reference for lines
sr = arcpy.Describe(dxPts).SpatialReference

# Generate lines-1
l1 = os.path.join(workingDir, 'l1.shp')
arcpy.BearingDistanceToLine_management(ltCSV, l1, "x", "y", "dist", "METERS", "bear1", "DEGREES", id_field = "id", spatial_reference = sr)
# Generate lines-1
l2 = os.path.join(workingDir, 'l2.shp')
arcpy.BearingDistanceToLine_management(ltCSV, l2, "x", "y", "dist", "METERS", "bear2", "DEGREES", id_field = "id", spatial_reference = sr)

# Copy l2 and l1 into new feature class
orthol = os.path.join(workingDir, 'ortho_lines.shp')

# Remove x, y, dist, bearing fields from L1 and L2
arcpy.DeleteField_management(l1, ["x", "y", "bear1"])
arcpy.DeleteField_management(l2, ["x", "y", "bear2"])

# Append L2 into L1
arcpy.Append_management(l2, l1, schema_type="TEST")

# Dissolve based on id field
arcpy.management.Dissolve(l1, orthol, "id")

arcpy.Delete_management(l1)
arcpy.Delete_management(l2)

# Remove all dx = 0 ortho_lines
with arcpy.da.UpdateCursor(orthol, "id") as uc:
    for row in uc:
        if str(row[0]).endswith("T_000"):
            uc.deleteRow()

In [20]:
# Use ortho_lines to split df_polys
orthop = os.path.join(workingDir, 'ortho_polys.shp')
arcpy.FeatureToPolygon_management("ortho_lines;Full_DF_Vector\df_polys", orthop)

# Remove all fields
fields = [f.name for f in arcpy.ListFields(orthop)]
fields.remove('FID')
fields.remove('Shape')
fields.remove('area_m2')
arcpy.DeleteField_management(orthop, fields)

# Add field for sub-poly ID
arcpy.AddField_management(orthop, "subpoly_id", "TEXT")

<Result 'Z:\\Auxiliary\\Project_Folders\\CS_Sandbox\\NCALM\\bulking\\v2\\points_run0\\ortho_polys.shp'>

In [21]:
# Create Spatial join to get orthol line id's into orthops
orthop_join = os.path.join(workingDir, 'ortho_polys_join.shp')

# Set up field map
fms = arcpy.FieldMappings()
fm = arcpy.FieldMap()
fm.addInputField(orthol, "id")
fm.mergeRule = "last"
id_name = fm.outputField
id_name.name = 'subpoly'
fm.outputField = id_name
fms.addFieldMap(fm)

arcpy.SpatialJoin_analysis(orthop, orthol, orthop_join, "JOIN_ONE_TO_ONE", match_option="WITHIN_A_DISTANCE", field_mapping = fms, search_radius="1 METERS")

# Add fields for zonal stats
arcpy.AddField_management(orthop_join, "net_total", "FLOAT")
arcpy.AddField_management(orthop_join, "eros", "FLOAT")
arcpy.AddField_management(orthop_join, "dep", "FLOAT")



<Result 'Z:\\Auxiliary\\Project_Folders\\CS_Sandbox\\NCALM\\bulking\\v2\\points_run0\\ortho_polys_join.shp'>

## In map view - check your analysis polygons - do they look ok?